# ___Les tests unitaires: cargo test___

---

> Cargo fournit un excellent outil pour mener les tests unitaires. Cela permet de s'assurer facilement qu'un bout de code ne représente pas de régression lors de modifications futures.

- Voici par exemple une fonction que vous voulez tester :

In [2]:
// Ici in retourne un entier positif sur 4 octets + 1
fn add_one(value: u32) -> u32 {
    value + 1
}

dbg!(add_one(1));

[src/lib.rs:10] add_one(1) = 2


### Comment s'assurer du bon fonctionnement de cette fonction add_one ?

- D'abord nous allons créer un nouveau programme nomme **add_one** avec `cargo new`

```
mordak@mordak-pc:~$ cargo new add_one
     Created binary (application) `add_one` package
```

- Le fichier **main.rs** sera ainsi:

```
fn add_one(value: u32) -> u32 {
    value + 1
}

fn main() {
}

#[cfg(test)]
mod add_one_test {
    use crate::add_one;

    #[test]
    fn check_add_one() {
        assert_eq!(add_one(1), 2)
    }
}
```

- La commande `cargo test` donne ceci:

```
mordak@mordak-pc:~/Documents/add_one$ cargo test
   Compiling add_one v0.1.0 (/home/mordak/Documents/add_one)
    Finished test [unoptimized + debuginfo] target(s) in 0.27s
     Running unittests src/main.rs (target/debug/deps/add_one-d72ba84568f68472)

running 1 test
test add_one_test::check_add_one ... ok

test result: ok. 1 passed; 0 failed; 0 ignored; 0 measured; 0 filtered out; finished in 0.00s
```

**La fonction main est vide pourtant une partie du programme s'est bel et bien exécutée, c'est le test de la fonction add_one.**

- Ci-dessous, le code de test:

In [10]:
#[cfg(test)]
mod add_one_test {
    use crate::add_one;

    #[test]
    fn check_add_one() {
        assert_eq!(add_one(1), 2)
    }
}

**Ici, nous avons plusieurs elements importants:**

>- **#[cfg(test)]** ne compile ce qui suit que lorsque l'ont appele **cargo test**, cfg est une directive préprocesseur.

In [11]:
#[cfg(target_os = "windows")]
fn main() {
    println!("This code will only be compiled on Windows");
}

>- **mod add_one_test** declare un module, ainsi tout ce qu'il contient ne sera compilé que pour les tests.  
>
>
>- **use crate::add_one;** signifie que l'on utilise la fonction add_one() de la crate principale, c’est-à-dire du fichier main.rs. cette fonction n'est en effet pas declaree directement dans le bloc de test. 
>
>
>- **#[test]** signifie 'ceci est un test'. On aurait pu avoir d'autres fonctions auxiliaires sans ce tag, elles ne seraient pas considérées comme des tests à part entière.  
>
>
>- **assert_eq!** est une macro Rust de la STD,  elle fonctionne comme une assertion en C, eq signifie équivalent et vérifie si l'expression de droite et de gauche sont égales. En outre, il existe aussi **assert_ne!** qui fait le contraire.

- Ainsi dans la forme la plus simple, nous pouvons écrire:

In [12]:
#[cfg(test)]
#[test]
fn check_add_one() {
    assert_eq!(add_one(1), 2)
}

**Cela fonctionne aussi, pourquoi ?**

> - Il n'y a pas besoin de module car il y a une seule fonction de test et rien d'autre, un module aurait été cependant nécessaire si nous étions passés par des fonctions auxiliaires du test ou des constantes etc...  
>
>
> - Comme l'on se situe dans la crate principale, le use crate::add_one est dorénavant inutile.

- Dans le cas où l'assertion serait fausse, nous aurions:

In [13]:
#[test]
fn check_add_one() {
    assert_eq!(add_one(1), 3)
}

```
mordak@mordak-pc:~/Downloads/add_one$ cargo test
   Compiling add_one v0.1.0 (/home/mordak/Downloads/add_one)
    Finished test [unoptimized + debuginfo] target(s) in 0.25s
     Running unittests src/main.rs (target/debug/deps/add_one-e13c01386e0c29ee)

running 1 test
test check_add_one ... FAILED

failures:

---- check_add_one stdout ----
thread 'check_add_one' panicked at 'assertion failed: `(left == right)`
  left: `2`,
 right: `3`', src/main.rs:14:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


failures:
    check_add_one

test result: FAILED. 0 passed; 1 failed; 0 ignored; 0 measured; 0 filtered out; finished in 0.00s

error: test failed, to rerun pass `--bin add_one`
```